
# Individual Household Electric Power Consumption — Análises em Python

Este notebook utiliza o conjunto de dados:  
<https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption>

Fonte direta dos arquivos (UCI):  
<https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip>

**Observações importantes**
- As medidas são registradas a cada minuto (amostragem de 1-min).
- Valores ausentes são representados por `?` no arquivo original.
- Unidade de **Global_active_power** e **Global_reactive_power**: quilowatts (kW).  
- Para converter potência (kW) em energia (kWh) em um período, utiliza-se a soma de `potência * (intervalo_em_horas)`. Com dados por minuto, `kWh = soma(kW) * (1/60)` ao longo do dia/mês.
- Algumas operações agregam por **dia** (ex.: consumo diário) e por **mês**.

Abaixo seguem as 20 tarefas solicitadas.



## Preparação do ambiente

Se o arquivo `household_power_consumption.txt` não estiver disponível localmente, a célula a seguir tenta baixá-lo e descompactá-lo automaticamente do repositório da UCI.  
> **Dica**: caso seu ambiente bloqueie downloads por Python, baixe manualmente o `.zip` do link acima e extraia o `.txt` na mesma pasta do notebook.


In [ ]:

import os, zipfile, urllib.request, io

DATA_ZIP_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip"
DATA_TXT = "household_power_consumption.txt"

if not os.path.exists(DATA_TXT):
    try:
        print("Baixando dataset da UCI...")
        with urllib.request.urlopen(DATA_ZIP_URL) as resp:
            data = resp.read()
        zf = zipfile.ZipFile(io.BytesIO(data))
        zf.extractall(".")
        print("Download e extração concluídos.")
    except Exception as e:
        print("Falha ao baixar automaticamente. Faça o download manual:", e)
else:
    print("Arquivo encontrado localmente:", DATA_TXT)


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from statsmodels.tsa.seasonal import seasonal_decompose

# Configurações gerais
pd.set_option("display.max_columns", None)



## 1) Carregar o dataset e exibir as 10 primeiras linhas


In [ ]:

# Leitura do arquivo (separador ';', valores ausentes '?')
df = pd.read_csv(
    "household_power_consumption.txt",
    sep=";",
    na_values=["?"],
    low_memory=False
)

# Exibe as 10 primeiras linhas
df.head(10)



## 2) Diferença entre **Global_active_power** e **Global_reactive_power**

- **Global_active_power (kW)**: potência ativa consumida pela residência — é a componente efetiva da potência que realiza trabalho útil (aquela que, integrada no tempo, resulta em **energia ativa (kWh)**).  
- **Global_reactive_power (kW)**: potência reativa associada aos campos elétricos e magnéticos de cargas indutivas/capacitivas. Ela não realiza trabalho útil líquido, mas circula entre a fonte e a carga, afetando o dimensionamento da instalação (correntes/cabos).

Na prática, a rede precisa transportar **ativa + reativa**, mas o consumo faturado em energia (kWh) é derivado da **potência ativa** integrada no tempo.


## 3) Verificar e quantificar valores ausentes

In [ ]:

missing_counts = df.isna().sum().sort_values(ascending=False)
missing_counts


## 4) Converter `Date` para `datetime` e criar coluna com o dia da semana

In [ ]:

# Combina Date e Time em um único datetime para análises temporais
# O dataset usa formato de data 'dd/mm/yyyy'; por isso, dayfirst=True.
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

# Converte Date para datetime
df['Date_dt'] = pd.to_datetime(df['Date'], format='%d/%m/%Y', errors='coerce')

# Cria coluna com o dia da semana (0=Segunda, 6=Domingo) e o nome do dia
df['Weekday'] = df['Date_dt'].dt.weekday
df['Weekday_name'] = df['Date_dt'].dt.day_name(locale='pt_BR') if hasattr(pd.Series.dt, 'day_name') else df['Date_dt'].dt.day_name()

df[['Date','Time','Date_dt','Weekday','Weekday_name']].head()


## 5) Filtrar 2007 e calcular média de consumo **diário** de `Global_active_power`

In [ ]:

# Seleciona apenas 2007
df_2007 = df[df['Date_dt'].dt.year == 2007].copy()

# Consumo diário (kWh) = soma de kW * 1/60 (min -> hora)
daily_energy_2007 = df_2007.groupby(df_2007['Date_dt'])['Global_active_power'].sum(min_count=1) * (1/60.0)

media_consumo_diario_2007 = daily_energy_2007.mean()
media_consumo_diario_2007


## 6) Gráfico de linha de `Global_active_power` em um único dia

In [ ]:

# Escolhemos um dia com boa cobertura no dataset: 2007-02-02 (presente no arquivo original).
dia_escolhido = pd.Timestamp('2007-02-02')

mask_day = df['Date_dt'] == dia_escolhido
day_data = df.loc[mask_day, ['Datetime','Global_active_power']].dropna()

plt.figure(figsize=(10,4))
plt.plot(day_data['Datetime'], day_data['Global_active_power'])
plt.title(f"Global_active_power ao longo do dia {dia_escolhido.date()}")
plt.xlabel("Tempo")
plt.ylabel("Potência ativa (kW)")
plt.tight_layout()
plt.show()


## 7) Histograma de `Voltage` e observações sobre a distribuição

In [ ]:

plt.figure(figsize=(6,4))
plt.hist(df['Voltage'].dropna(), bins=50, alpha=0.8)
plt.title("Distribuição de Voltage")
plt.xlabel("Tensão (V)")
plt.ylabel("Frequência")
plt.tight_layout()
plt.show()

# Observação sugerida (em Markdown abaixo): geralmente distribuição aproximadamente unimodal,
# centrada por volta de ~240V, com cauda moderada dependendo de outliers/horários.


## 8) Consumo médio **por mês** em todo o período

In [ ]:

# Energia diária (kWh) em todo o período
daily_energy_all = df.groupby(df['Date_dt'])['Global_active_power'].sum(min_count=1) * (1/60.0)

# Média mensal do consumo (kWh/dia) calculada como média das energias diárias dentro de cada mês
monthly_avg = daily_energy_all.groupby([daily_energy_all.index.year, daily_energy_all.index.month]).mean()
monthly_avg = monthly_avg.rename_axis(index=['Ano','Mes']).to_frame('Consumo_medio_kWh_dia')
monthly_avg.head(12)


## 9) Dia com **maior consumo de energia ativa global**

In [ ]:

max_day = daily_energy_all.idxmax()
max_value = daily_energy_all.max()
max_day, float(max_value)


## 10) Consumo médio em **dias de semana** vs **finais de semana**

In [ ]:

daily_df = daily_energy_all.to_frame('kWh').reset_index().rename(columns={'index':'Date'})
daily_df['Weekday'] = daily_df['Date'].dt.weekday
daily_df['is_weekend'] = daily_df['Weekday'] >= 5

consumo_semana = daily_df.loc[~daily_df['is_weekend'], 'kWh'].mean()
consumo_fimsemana = daily_df.loc[daily_df['is_weekend'], 'kWh'].mean()
consumo_semana, consumo_fimsemana


## 11) Correlação entre `Global_active_power`, `Global_reactive_power`, `Voltage`, `Global_intensity`

In [ ]:

corr_cols = ['Global_active_power','Global_reactive_power','Voltage','Global_intensity']
corr_df = df[corr_cols].astype(float).corr(method='pearson')
corr_df


## 12) Criar `Total_Sub_metering = Sub_metering_1 + Sub_metering_2 + Sub_metering_3`

In [ ]:

sub_cols = ['Sub_metering_1','Sub_metering_2','Sub_metering_3']
df['Total_Sub_metering'] = df[sub_cols].sum(axis=1, min_count=1)
df[['Total_Sub_metering'] + sub_cols].head()


## 13) Algum mês em que `Total_Sub_metering` **ultrapassa** a média de `Global_active_power`?

In [ ]:

# Calcula médias mensais (em kW) comparáveis entre si
df_monthly = df.set_index('Datetime').resample('M')[['Global_active_power','Total_Sub_metering']].mean()

cond = df_monthly['Total_Sub_metering'] > df_monthly['Global_active_power']
excedentes = df_monthly[cond]
excedentes


## 14) Série temporal de `Voltage` para 2008

In [ ]:

df_2008 = df[(df['Date_dt'].dt.year == 2008)].copy()
ts_2008 = df_2008.set_index('Datetime')['Voltage'].dropna()

plt.figure(figsize=(10,4))
plt.plot(ts_2008.index, ts_2008.values)
plt.title("Voltage ao longo de 2008")
plt.xlabel("Tempo")
plt.ylabel("Tensão (V)")
plt.tight_layout()
plt.show()


## 15) Comparação de consumo entre **verão** e **inverno** (Hemisfério Norte)

In [ ]:

# Verão (JJA) = Junho(6), Julho(7), Agosto(8)
# Inverno (DJF) = Dez(12), Jan(1), Fev(2)
daily_df = daily_energy_all.to_frame('kWh').reset_index().rename(columns={'index':'Date'})
daily_df['month'] = daily_df['Date'].dt.month

summer = daily_df[daily_df['month'].isin([6,7,8])]['kWh'].mean()
winter = daily_df[daily_df['month'].isin([12,1,2])]['kWh'].mean()
summer, winter


## 16) Amostragem aleatória de 1% e comparação da distribuição de `Global_active_power`

In [ ]:

# Amostra 1% das linhas não nulas
gact = df['Global_active_power'].dropna().astype(float)
sample = gact.sample(frac=0.01, random_state=42)

plt.figure(figsize=(6,4))
plt.hist(gact, bins=60, density=True, alpha=0.5, label='Base completa')
plt.hist(sample, bins=60, density=True, alpha=0.5, label='Amostra 1%')
plt.title("Distribuições de Global_active_power")
plt.xlabel("kW")
plt.ylabel("Densidade")
plt.legend()
plt.tight_layout()
plt.show()

# Comparação numérica simples (média e desvio)
gact.mean(), gact.std(), sample.mean(), sample.std()


## 17) Normalização Min-Max das variáveis numéricas principais

In [ ]:

num_cols = ['Global_active_power','Global_reactive_power','Voltage','Global_intensity','Sub_metering_1','Sub_metering_2','Sub_metering_3','Total_Sub_metering']
num_df = df[num_cols].astype(float)

scaler = MinMaxScaler()
scaled_vals = scaler.fit_transform(num_df.dropna())

scaled_df = pd.DataFrame(scaled_vals, columns=num_cols, index=num_df.dropna().index)
scaled_df.head()


## 18) K-means para segmentar **dias** em 3 grupos de consumo elétrico

In [ ]:

# Cria features diárias para clusterização
grp = df.set_index('Datetime').groupby(pd.Grouper(freq='D'))
daily_features = pd.DataFrame({
    'energy_kWh': grp['Global_active_power'].sum(min_count=1) * (1/60.0),
    'mean_active_kW': grp['Global_active_power'].mean(),
    'max_active_kW': grp['Global_active_power'].max(),
    'mean_reactive_kW': grp['Global_reactive_power'].mean(),
    'mean_voltage': grp['Voltage'].mean(),
    'mean_intensity': grp['Global_intensity'].mean(),
    'sub_total_mean': grp['Total_Sub_metering'].mean()
}).dropna()

X = daily_features[['energy_kWh','mean_active_kW','max_active_kW','mean_reactive_kW','mean_voltage','mean_intensity','sub_total_mean']].values

kmeans = KMeans(n_clusters=3, n_init=20, random_state=42)
labels = kmeans.fit_predict(X)
daily_features['cluster'] = labels

# Centroides (interpretação)
centroids = pd.DataFrame(kmeans.cluster_centers_, columns=['energy_kWh','mean_active_kW','max_active_kW','mean_reactive_kW','mean_voltage','mean_intensity','sub_total_mean'])
cluster_sizes = daily_features['cluster'].value_counts().sort_index()

centroids, cluster_sizes


## 19) Decomposição de série temporal (6 meses) para `Global_active_power`

In [ ]:

# Usaremos série diária média de Global_active_power e um intervalo de 6 meses (ex.: 2007-01 a 2007-06).
periodo = (pd.Timestamp('2007-01-01'), pd.Timestamp('2007-06-30'))

daily_mean_active = df.set_index('Datetime').loc[periodo[0]:periodo[1], 'Global_active_power'].resample('D').mean()

# seasonal_decompose requer um período sazonal; para diário podemos considerar sazonalidade semanal (~7)
result = seasonal_decompose(daily_mean_active.dropna(), model='additive', period=7)

fig = result.plot()
fig.set_size_inches(10,8)
plt.tight_layout()
plt.show()


## 20) Regressão linear simples: prever `Global_active_power` a partir de `Global_intensity`

In [ ]:

data_lr = df[['Global_active_power','Global_intensity']].dropna().astype(float)

X = data_lr[['Global_intensity']].values
y = data_lr['Global_active_power'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

coef = float(model.coef_[0])
intercept = float(model.intercept_)

coef, intercept, mae, rmse, r2



### Notas de interpretação (guias rápidos)

- **(7) Voltage**: normalmente apresenta distribuição aproximadamente unimodal, com valores concentrados ao redor de ~240 V, podendo haver caudas e pequenas variações conforme horários.
- **(9) Dia de maior consumo**: calculado pela **energia diária (kWh)** a partir de `Global_active_power`. Esse valor pode indicar dias com maior uso de eletrodomésticos ou condições sazonais.
- **(10) Semana vs. Fim de semana**: diferenças podem refletir padrões de permanência em casa, rotinas de trabalho e uso de equipamentos.
- **(11) Correlações**: espera-se correlação positiva entre `Global_active_power` e `Global_intensity`, já que intensidade de corrente tende a aumentar com a potência ativa.
- **(13) Sub-metering vs. Global**: `Total_Sub_metering` mede apenas três subcircuitos; é comum que **não** supere a média de `Global_active_power` mensal, pois há consumo não medido por esses submedidores.
- **(18) K-means**: clusters podem separar dias de **baixo**, **médio** e **alto** consumo. Interprete pelos centroides e tamanhos dos grupos.
- **(19) Decomposição**: em dados diários, a sazonalidade semanal (período ~7) é útil para revelar padrões por dia da semana.
- **(20) Regressão**: avalie MAE/RMSE/R²; relação linear simples capta parte, mas não toda a variabilidade.



# PARTE 2 — Exercícios adicionais

Neste arquivo adicionamos as tarefas 21 a 25 solicitadas, que cobrem reamostragem horária, autocorrelação, PCA, visualização de clusters no espaço PCA e comparação entre regressão linear e polinomial.



## 21) Séries temporais por hora
- Converta `Date` e `Time` em índice `Datetime`.
- Reamostre os dados em intervalos de 1 hora, calculando a **média** de `Global_active_power`.
- Identifique os horários (hora do dia) de maior consumo médio ao longo do dia.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Leitura (assume que o arquivo foi extraído previamente)
df = pd.read_csv('household_power_consumption.txt', sep=';', na_values=['?'], low_memory=False)

# Criar Datetime
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
df = df.dropna(subset=['Datetime']).set_index('Datetime')

# Converter colunas numéricas
for col in ['Global_active_power','Global_reactive_power','Voltage','Global_intensity','Sub_metering_1','Sub_metering_2','Sub_metering_3']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Reamostrar por hora (média de Global_active_power)
hourly = df['Global_active_power'].resample('H').mean()

# Identificar horário do dia com maior consumo médio ao longo do período
# Calculamos a média por hora-of-day (0-23)
hour_of_day = hourly.groupby(hourly.index.hour).mean().sort_values(ascending=False)
hour_of_day



## 22) Autocorrelação do consumo
- Use a série temporal de `Global_active_power` reamostrada por hora.
- Calcule a autocorrelação para lags de 1h, 24h e 48h.
- Pergunta: existem padrões repetidos diariamente?


In [ ]:

# Garantir série horária contínua com frequência horária
hourly_full = hourly.asfreq('H')

# Autocorrelações
autocorr_1h = hourly_full.autocorr(lag=1)
autocorr_24h = hourly_full.autocorr(lag=24)
autocorr_48h = hourly_full.autocorr(lag=48)

autocorr_1h, autocorr_24h, autocorr_48h



## 23) Redução de dimensionalidade com PCA
- Selecionar `Global_active_power`, `Global_reactive_power`, `Voltage` e `Global_intensity`.
- Aplicar PCA para reduzir para 2 componentes principais.
- Analisar a variância explicada por cada componente.


In [ ]:

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pca_df = df[['Global_active_power','Global_reactive_power','Voltage','Global_intensity']].dropna().astype(float)

# Padronizar antes do PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(pca_df)

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

explained_variance = pca.explained_variance_ratio_
explained_variance



## 24) Visualização de clusters no espaço PCA
- Combine o PCA com K-Means (3 clusters).
- Plote os pontos no espaço dos 2 componentes principais, colorindo por cluster.
- Pergunta: os grupos se separam de forma clara?


In [ ]:

from sklearn.cluster import KMeans

# Aplicar KMeans sobre os componentes PCA
kmeans_pca = KMeans(n_clusters=3, n_init=20, random_state=42)
labels_pca = kmeans_pca.fit_predict(X_pca)

import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))
scatter = plt.scatter(X_pca[:,0], X_pca[:,1], c=labels_pca, s=10, cmap='tab10', alpha=0.6)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('PCA (2 componentes) com K-Means (3 clusters)')
plt.grid(alpha=0.3)
plt.show()

# Centroides dos clusters no espaço PCA
centroids_pca = kmeans_pca.cluster_centers_
centroids_pca



## 25) Regressão polinomial vs linear
- Modelar `Global_active_power` em função de `Voltage`.
- Comparar Regressão Linear Simples com Regressão Polinomial (grau 2).
- Analisar RMSE e a curva ajustada.


In [ ]:

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

# Preparar dados (remoção de NA)
reg_df = df[['Voltage','Global_active_power']].dropna().astype(float)

# Amostragem para acelerar (opcional)
reg_sample = reg_df.sample(frac=0.2, random_state=42) if len(reg_df) > 200000 else reg_df.copy()

X = reg_sample[['Voltage']].values
y = reg_sample['Global_active_power'].values

# Linear
lr = LinearRegression()
lr.fit(X, y)
y_pred_lin = lr.predict(X)
rmse_lin = np.sqrt(mean_squared_error(y, y_pred_lin))

# Polinomial grau 2
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)
lr_poly = LinearRegression()
lr_poly.fit(X_poly, y)
y_pred_poly = lr_poly.predict(X_poly)
rmse_poly = np.sqrt(mean_squared_error(y, y_pred_poly))

rmse_lin, rmse_poly



### Observações e interpretação rápida

- **21 (hora de maior consumo)**: a agregação por hora-of-day revela horários com maior média — normalmente horários noturnos/manhãs dependendo do padrão local.  
- **22 (autocorrelação)**: autocorrelação alta em lag=24h indica padrão diário repetido; lag=48h reforça periodicidade.  
- **23 (PCA)**: verifique `explained_variance` para ver quanto dos dados os dois PCs capturam; valores maiores que ~0.6/0.7 indicam boa compressão.  
- **24 (visualização)**: a separação visual dos clusters pode ser clara ou sobreposta; verifique centroides e distribuição para interpretar (ex.: cluster 0 = dias de alto consumo).  
- **25 (regressão)**: se a relação tensão → potência for fraca, RMSE permanecerá alto e o ganho do polinômio será pequeno.
